# Practice 2.1 (Recurrent Neural Networks)

Authors:

1. Ovidio Manteiga Moar
1. Carlos Villar Martínez

### Introduction

For this first part of the practise we are using the Walmart Sales Dataset of 45 Stores. The file has information about the Weekly Sales of 45 stores for the year 2010-2012. With this information the goal is to create and train a model capable of predicting the sales of the next three weeks.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Reshape, SimpleRNN, Dense

In [2]:
#Returns a numpy array with size nrows x ncolumns-1. nrows and ncolums are the rows and columns of the dataset
#the Date column is skipped (ncolumns-1)
def readData(fname):
    with open(fname) as f:
        fileData = f.read()
  
    lines = fileData.split("\n")
    header = lines[0].split(",")
    lines = lines[1:] 
    #print(header) 
    #print("Data rows: ", len(lines))

    rawData = np.zeros((len(lines), len(header)-1)) #skip the Date column

    for i, aLine in enumerate(lines):       
        splittedLine = aLine.split(",")[:]
        rawData[i, 0] = splittedLine[0]
        rawData[i, 1:] = [float(x) for x in splittedLine[2:]] 

    return rawData

In [3]:
#Returns the train and test data, normalized. It also returns the standard deviation of Weekly_Sales
#Each list has a size equal to the number of stores
#For each store there is a list of size trainNSaples (testNSamples) x nColums-1 (the store id is skipped)
#Columns: Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
def splitTrainTest(rawData, testPercent):

    listStore = np.unique(rawData[:, 0])
    trainNSamples = np.zeros(len(listStore))
    
    for i, storeId in enumerate(listStore):
        trainNSamples[i] = np.count_nonzero(rawData[:, 0] == storeId)
    trainNSamples = np.floor((1-testPercent) *  trainNSamples)

    tmpTrain = np.zeros((int(np.sum(trainNSamples)), len(rawData[0])))

    store = -1
    counter = 0
    counterTrain = 0
    storeDict = dict(zip(listStore, trainNSamples))
    for i, aLine in enumerate(rawData):
        if store != aLine[0]:
            store = int(aLine[0])
            counter = 0
        if(counter < storeDict.get(store)):
            tmpTrain[counterTrain] = rawData[i][:]
            counterTrain += 1
            counter += 1

    meanData = tmpTrain.mean(axis=0)
    stdData = tmpTrain.std(axis=0)
    rawNormData = (rawData - meanData) / stdData

    allTrain = list()
    allTest = list()
    store = -1
    counter = 0
    for i, aLine in enumerate(rawNormData):
        splittedLine = [float(x) for x in aLine[1:]] #skip store id
        if store != rawData[i][0]:
            if i != 0:
                allTrain.append(storeDataTrain)
                allTest.append(storeDataTest)
            store = int(rawData[i][0])
            storeDataTrain = list()
            storeDataTest = list()
            counter = 0

        if(counter < storeDict.get(store)):
            storeDataTrain.append(splittedLine)
            counter += 1
        else:
            storeDataTest.append(splittedLine)

        if i == len(rawNormData)-1:
            allTrain.append(storeDataTrain)
            allTest.append(storeDataTest)

    return allTrain, allTest, stdData[1] #std of wSales

In [4]:
#generates a time series given the input and ouput data, the sequence length and the batch size
#seqLength is the number of weeks (observations) of data to be used as input
#the target will be the weekly sales in 2 weeks
def generateTimeSeries(data, wSales, seqLength, batchSize):   
    sampling_rate = 1 #keep all the data points 
    weeksInAdvance = 3
    delay = sampling_rate * (seqLength + weeksInAdvance - 1) #the target will be the weekly sales in 2 weeks
    
    dataset = keras.utils.timeseries_dataset_from_array(
        data[:-delay],
        targets=wSales[delay:],
        sampling_rate=sampling_rate,
        sequence_length=seqLength,
        shuffle=True,
        batch_size=batchSize,
        start_index=0)
    
    return dataset


In [5]:
def printTimeSeriesList(theList):
    print('list length', len(theList))
    print('First element')
    input, target = theList[0]
    print([float(x) for x in input.numpy().flatten()], [float(x) for x in target.numpy().flatten()])
    print('Last element')
    input, target = theList[-1]
    print([float(x) for x in input.numpy().flatten()], [float(x) for x in target.numpy().flatten()])

In [6]:
#returns the training and test time series
#it also returns the standard deviation of Weekly_Sales, and the number of input features
def generateTrainTestData(fileName, testPercent, seqLength, batchSize):
    rawData = readData(os.path.join(fileName))
    allTrain, allTest, stdSales = splitTrainTest(rawData, testPercent)
    
    for i in range(len(allTrain)):
        tmp_train = generateTimeSeries(np.array(allTrain[i]), np.array(allTrain[i])[:,0], seqLength, batchSize)
        tmp_test = generateTimeSeries(np.array(allTest[i]), np.array(allTest[i])[:,0], seqLength, batchSize)

        if i == 0:
            train_dataset = tmp_train
            test_dataset = tmp_test
        else:
            train_dataset = train_dataset.concatenate(tmp_train)
            test_dataset = test_dataset.concatenate(tmp_test)
    
    return train_dataset, test_dataset, stdSales, np.shape(allTrain)[2]

# The model

In the following cells the starts the design of ur model. First of all we define the percentage of samples that are going to be part of the test set (as in the first practice we are going to use the test set as validation set), we decided to use a common split of 20% as test set and a 80% as training set. We also decided to change the sequence lenght from 8 to 12 as it gave us better results. After this we print the value of the sales standard deviation, as we will need it in the future in order to calculate the denormalized mae.

In [72]:
testPercent = 0.2
seqLength = 12
batchSize = 1
nFeatures = 6

In [73]:
trainData, testData, stdSales, nFeatures = generateTrainTestData("walmart-sales-dataset-of-45stores.csv",
    testPercent, seqLength, batchSize)
print(f"STD(sales) = {stdSales}")

STD(sales) = 571854.7800576452


The following cell is the most important one as all the layers of the model are ordered and defined there. We tried a lot of different things, for example using GRUs, RNNs or dropout, some of them obtained good results but te best one that we found was using a single LSTM layer.

First of all we defined the input layer, in shape section we can write None instead of `seqLength` but, if all of our sequences have the same lenght it is recomended to specify the full shape as it may help to unlock some performance optimizations.

After defining the input layer we present the LSTM layer, which is the recurrent layer of our model. The output of the last LSTM unit is the only one considered for the prediction, hence the `return_sequences` attribute is set to `False`. This is an example of a many to one RNN.

Since the dimensionality of the output will be 128 (as the number of units), a dense layer with a single neuron is added to produce a single real value as output. It has only one output neuron that represents the prediction and, since it is a regression problem we do not need an activation function.

In [74]:
inputs = keras.Input(shape=(seqLength, nFeatures))
x = layers.LSTM(128, activation='tanh', return_sequences=False)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [75]:
model.summary()

Model: "model_205"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_209 (InputLayer)      [(None, 12, 6)]           0         
                                                                 
 lstm_97 (LSTM)              (None, 128)               69120     
                                                                 
 dense_205 (Dense)           (None, 1)                 129       
                                                                 
Total params: 69,249
Trainable params: 69,249
Non-trainable params: 0
_________________________________________________________________


In [76]:
callbacks = [keras.callbacks.ModelCheckpoint("jena_gru.keras")]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(trainData, epochs=20, validation_data=testData, callbacks=callbacks)

Epoch 1/20
4005/4005 [==============================] - 49s 11ms/step - loss: 0.0965 - mae: 0.1825 - val_loss: 0.2062 - val_mae: 0.3506
Epoch 2/20
4005/4005 [==============================] - 52s 13ms/step - loss: 0.0466 - mae: 0.1378 - val_loss: 0.1567 - val_mae: 0.3043
Epoch 3/20
4005/4005 [==============================] - 49s 12ms/step - loss: 0.0396 - mae: 0.1277 - val_loss: 0.1486 - val_mae: 0.2979
Epoch 4/20
4005/4005 [==============================] - 52s 13ms/step - loss: 0.0368 - mae: 0.1220 - val_loss: 0.1078 - val_mae: 0.2506
Epoch 5/20
4005/4005 [==============================] - 69s 17ms/step - loss: 0.0354 - mae: 0.1203 - val_loss: 0.0887 - val_mae: 0.2297
Epoch 6/20
4005/4005 [==============================] - 62s 15ms/step - loss: 0.0328 - mae: 0.1152 - val_loss: 0.0556 - val_mae: 0.1799
Epoch 7/20
4005/4005 [==============================] - 52s 13ms/step - loss: 0.0337 - mae: 0.1137 - val_loss: 0.0637 - val_mae: 0.1962
Epoch 8/20
4005/4005 [==========================

In [77]:
min_val_mae = min(history.history['val_mae'])
std = stdSales
denormalized_MAE = stdSales * min_val_mae
print(f'Denormalized MAE = {denormalized_MAE} (val_mae = {min_val_mae})')

Denormalized MAE = 67624.53608371985 (val_mae = 0.11825473606586456)


In the previous cell we calculate the denormalized MAE using the minimum validation MAE result obtained for the validation set, since that model snapshot is stored with the callback and it is the one to choose.

# Conclusions

In this first part of the practice, we have worked with something quite different from what we worked in the previous practice, we stopped working with images to start working with a dataset containing data from sales and try to predict those sales. This is a problem that needs a complete different approach. We were also asked to use at least one recurrent layer which was not asked in the previous practice. 

First of all, we had to try a lot of different models to find the one that offered the best results, we found it when we tried a LSTM layer with the configuration that can be seen in the model section. We also realised that for this dataset a dropout or another kind of regularization layer did not worked really well, the same happened when we tried to stack several recurrent layers (even though we found some promising results with those configurations). 

We also tried different sequence length values and we found that with bigger ones we obtained better results, being 12 the most reasonable one. The single layer GRU with the same number of units could also be considered for performance reasons, since it performed close to the LSTM.

Another thing that we tried were different activation functions and optimizers, but without any relevant improvements.

Increasing the batch size provides better results and faster compared to 1, which causes a lot of fluctuation within each epic and a slower convergence. We used the batching for evaluating different models although from the description of the assignment we understood that parameter could not be changed in the final model evaluation.

Adding RNN layers could help in problems where there are more complex relationships between the features and the output, but in this case, after testing with multiple configurations, it works worse in general, so a simpler model is preferred.

Finally, the best result obtained was a val_mae that oscilated in values of 0,11 and 0,12 which is translated into values of approximately 67000 for the denormalized MAE. In the practice statement, it is said that a good result is a denormalized MAE of 68000 or below, so we can say that our result is quite satisfying.



# Appendix: model/hyperparameters performance comparison

In [67]:
def generate_data(seqLength, batchSize):
    testPercent = 0.2
    nFeatures = 6
    csv = "walmart-sales-dataset-of-45stores.csv"
    trainData, testData, stdSales, nFeatures = generateTrainTestData(csv, testPercent, seqLength, batchSize)
    return trainData, testData, stdSales, nFeatures

def create_RNN(units, layer, seqLength, nFeatures):
    inputs = keras.Input(shape=(seqLength, nFeatures))
    x = layer(units, activation='tanh', return_sequences=False)(inputs)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    name = "1-layer RNN ({layer_type}) with {units} units".format(layer_type=layer.__name__, units=units)
    return model, name

def create_double_RNN(units, layer, seqLength, nFeatures):
    inputs = keras.Input(shape=(seqLength, nFeatures))
    x = layer(units, activation='tanh', return_sequences=True)(inputs)
    x = layer(units//2, activation='tanh', return_sequences=False)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    name = "2-layer RNN ({layer_type}) with {units} units".format(layer_type=layer.__name__, units=units)
    return model, name

def create_simpleRNN(units, seqLength, nFeatures):
    return create_RNN(units, layers.SimpleRNN, seqLength, nFeatures)

def create_double_simpleRNN(units, seqLength, nFeatures):
    return create_double_RNN(units, layers.SimpleRNN, seqLength, nFeatures)

def create_GRU(units, seqLength, nFeatures):
    return create_RNN(units, layers.GRU, seqLength, nFeatures)

def create_double_GRU(units, seqLength, nFeatures):
    return create_double_RNN(units, layers.GRU, seqLength, nFeatures)

def create_LSTM(units, seqLength, nFeatures):
    return create_RNN(units, layers.LSTM, seqLength, nFeatures)

def create_double_LSTM(units, seqLength, nFeatures):
    return create_double_RNN(units, layers.LSTM, seqLength, nFeatures)

def train_and_evaluate(model, name, trainData, testData, stdSales, epochs=5, verbose=1):
    # Train
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    history = model.fit(trainData, epochs=epochs, validation_data=testData, verbose=verbose)
    # Evaluate
    min_val_mae = min(history.history['val_mae'])
    std = stdSales
    denormalized_MAE = stdSales * min_val_mae
    print(f'Model = {name}')
    print(f'Denormalized MAE = {denormalized_MAE} (val_mae = {min_val_mae})')
    return name, denormalized_MAE, min_val_mae



In [69]:
# Generate data
trainData, testData, stdSales, nFeatures = generate_data(seqLength=12, batchSize=64)
# Create model
model, name = create_simpleRNN(128, 12, nFeatures)
print(name)
# Train model
train_and_evaluate(model, name, trainData, testData, stdSales, epochs=5)

1-layer RNN (SimpleRNN) with 128 units
Epoch 1/5
90/90 [==============================] - 8s 79ms/step - loss: 0.3257 - mae: 0.3877 - val_loss: 0.1449 - val_mae: 0.3297
Epoch 2/5
90/90 [==============================] - 6s 61ms/step - loss: 0.1322 - mae: 0.2690 - val_loss: 0.0472 - val_mae: 0.1734
Epoch 3/5
90/90 [==============================] - 6s 65ms/step - loss: 0.0961 - mae: 0.2300 - val_loss: 0.0377 - val_mae: 0.1461
Epoch 4/5
90/90 [==============================] - 6s 65ms/step - loss: 0.0737 - mae: 0.2037 - val_loss: 0.0398 - val_mae: 0.1618
Epoch 5/5
90/90 [==============================] - 6s 64ms/step - loss: 0.0622 - mae: 0.1839 - val_loss: 0.0411 - val_mae: 0.1561
Model = 1-layer RNN (SimpleRNN) with 128 units
Denormalized MAE = 83566.3465094285 (val_mae = 0.14613211154937744)


('1-layer RNN (SimpleRNN) with 128 units',
 83566.3465094285,
 0.14613211154937744)

In [70]:
def create_all_models(units, seqLength, nFeatures):
    models = []
    models += [create_simpleRNN(units, seqLength, nFeatures)]
    models += [create_double_simpleRNN(units, seqLength, nFeatures)]
    models += [create_GRU(units, seqLength, nFeatures)]
    models += [create_double_GRU(units, seqLength, nFeatures)]
    models += [create_LSTM(units, seqLength, nFeatures)]
    models += [create_double_LSTM(units, seqLength, nFeatures)]
    return models

bestMAE = 1.0
best = None
for length in [4, 8, 12]:
    trainData, testData, stdSales, nFeatures = generate_data(seqLength=length, batchSize=64)
    for units in [32, 64, 128]:
        models = create_all_models(units, length, nFeatures=6)
        for model in models:
            modelName = model[1] + "(length = {length})".format(length=length)
            name, dMAE, valMAE = train_and_evaluate(model[0], modelName, trainData, testData, stdSales, epochs=5, verbose=0)
            if valMAE < bestMAE:
                bestMAE = valMAE
                best = (dMAE, name, length)

print("BEST MODEL")
print(best)

Model = 1-layer RNN (SimpleRNN) with 32 units(length = 4)
Denormalized MAE = 120342.58268036958 (val_mae = 0.21044255793094635)
Model = 2-layer RNN (SimpleRNN) with 32 units(length = 4)
Denormalized MAE = 111683.50151940108 (val_mae = 0.19530045986175537)
Model = 1-layer RNN (GRU) with 32 units(length = 4)
Denormalized MAE = 76374.08788951811 (val_mae = 0.13355503976345062)
Model = 2-layer RNN (GRU) with 32 units(length = 4)
Denormalized MAE = 78637.1577692012 (val_mae = 0.1375124603509903)
Model = 1-layer RNN (LSTM) with 32 units(length = 4)
Denormalized MAE = 79341.6733105584 (val_mae = 0.13874444365501404)
Model = 2-layer RNN (LSTM) with 32 units(length = 4)
Denormalized MAE = 76848.96291024641 (val_mae = 0.13438545167446136)
Model = 1-layer RNN (SimpleRNN) with 64 units(length = 4)
Denormalized MAE = 104328.1196486688 (val_mae = 0.18243813514709473)
Model = 2-layer RNN (SimpleRNN) with 64 units(length = 4)
Denormalized MAE = 123425.25678268241 (val_mae = 0.21583321690559387)
Model 